Creazione BASE DATI dataset finale

Dependencies

In [1]:
import pandas as pd
import numpy as np 

Datasets

In [22]:
p05 = pd.read_csv("./cig_pnrr_anac.csv/cig_pnrr_anac.csv",sep=",")
p03 = pd.read_csv("./datasets/misurepremiali-pnrrpnc_csv.csv",delimiter=';')
p04 = pd.read_csv("./datasets/indicatori-pnrrpnc_csv.csv",delimiter=';')

C:\Users\Alessandra\AppData\Local\Temp\ipykernel_2788\1520190168.py:1: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  p05 = pd.read_csv("./cig_pnrr_anac.csv/cig_pnrr_anac.csv",sep=",")


In [23]:
p01 = pd.read_csv("./datasets/cup_csv.csv",delimiter=';',encoding = "ISO-8859-1")
p02 = pd.read_excel("./datasets/CUP_PNRR_2023_12_08.xlsx")
comuni = pd.read_csv("./datasets/Elenco-comuni-italiani.csv",delimiter=';',encoding = "ISO-8859-1")
missioni_map = pd.read_json("./missioni_pnrr.json", orient="index")

Pre-processing dei dataset

In [24]:
# preprocessing
p05 = p05.rename(columns={'cig':'CIG'})
p03 = p03.rename(columns={'cig':'CIG'})
p04 = p04.rename(columns={'cig':'CIG'})

In [25]:
# pre-processing p03
# add colonna 'CATEGORIA'
def conditions(s):
    if s['cod_mis_premiale'] in [1,2,9]:
        return "GENERALE"
    elif s['cod_mis_premiale'] in [3,8,12]:
        return "DISABILI"
    elif s['cod_mis_premiale'] in [4,6,10]:
        return "GENERE"
    elif s['cod_mis_premiale'] in [5,7,11]:
        return "ETÀ"
    else:
        return 0
    
p03['CATEGORIA'] = p03.apply(conditions,axis=1)

In [26]:
# pre-processing p05 pnew
# add colonna 'CLASSE_IMPORTO' 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
def map_importo(importo):
    if importo <= 100000:
        return 'BASSA'
    elif importo <= 1000000:
        return 'MEDIA'
    else:
        return 'ALTA'
    
p05['CLASSE_IMPORTO'] = p05['importo_complessivo_gara'].apply(map_importo)
# add colonna 'REGIONI'
prov_reg = comuni[['Denominazione Regione','Denominazione dell\'Unità territoriale sovracomunale \n(valida a fini statistici)']]
prov_reg = prov_reg.rename(columns={'Denominazione dell\'Unità territoriale sovracomunale \n(valida a fini statistici)':'Denominazione sovracomunale','Denominazione Regione':'Regione'})
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].replace(to_replace=r"Valle d'Aosta/Vallée d'Aoste", value='Aosta', regex=False)
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].replace(to_replace=r"Bolzano/Bozen", value='Bolzano',regex=False)
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].str.upper()
prov_reg=prov_reg.drop_duplicates()
p05=p05.merge(prov_reg,left_on='provincia', right_on='Denominazione sovracomunale',how='left')
p05=p05.drop_duplicates()
#p05_new = p05[['CIG','importo_complessivo_gara','CLASSE_IMPORTO', 'luogo_istat', 'provincia', 'Denominazione sovracomunale','Regione','data_pubblicazione']]

In [27]:
p05['importo_complessivo_gara'].dtype
p02['COSTO_PROGETTO'].dtype
p02['IMPORTO_FINANZIAMENTO'].dtype

dtype('float64')

In [28]:
# pre-processing p02
# add colonna 'CODICE COMPONENTE', 'CODICE MISSIONE', 'COMPONENTE', 'INVESTIMENTO' 
def map_comp(tematica):
    if tematica.count('-')==1:
        componente = tematica[6:tematica.find('-')-1]
    else:
        componente = tematica[6:tematica.find('I')-4]
    return componente

def map_investimento(tematica):
    # con numeri
    if tematica.count('-')==1:
        investimento = tematica[tematica.find('-')+2:]
    else:
        first = tematica.find('-')+1
        tematica = tematica[first:]
        investimento = tematica[tematica.find('-')+2:]
    return investimento

p02["CODICE_COMPONENTE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:4])
p02["CODICE_MISSIONE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:2])
p02["COMPONENTE"] = p02['NOME_TEMATICA'].apply(map_comp)
p02["INVESTIMENTO"] = p02['NOME_TEMATICA'].apply(map_investimento)
# add colonna 'MISSIONE'
missioni_map.columns = ["MISSIONE"]
p02 = p02.merge(missioni_map,left_on="CODICE_MISSIONE", right_on=missioni_map.index, how='inner')
p02 = p02.drop_duplicates()

joins

In [30]:
# left join p5 e p3
leftnew_3 = p05.merge(p03, on='CIG',how='left')
leftnew_3 = leftnew_3.drop_duplicates()
print(leftnew_3.columns)
print(leftnew_3.shape)

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'DURATA_PREVIST

In [31]:
# left join il risultato con p4
leftnew_34 = leftnew_3.merge(p04, on='CIG', how='left')
leftnew_34 = leftnew_34.drop_duplicates()
print(leftnew_34.columns)
print(leftnew_34.shape)

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'DURATA_PREVIST

In [32]:
# left join il risultato con p1
leftnew_341 = leftnew_34.merge(p01, on='CIG', how='left')
leftnew_341 = leftnew_341.drop_duplicates()
print(leftnew_341.columns)
print(leftnew_341.shape)

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'DURATA_PREVIST

In [33]:
# left join il risultato con p2
leftnew_3412 = leftnew_341.merge(p02,how='left',left_on='CUP',right_on='CODICE_CUP')
leftnew_3412 = leftnew_3412.drop_duplicates()
print(leftnew_3412.columns)
print(leftnew_3412.shape)

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato',
       ...
       'DATA_GENERAZIONE_COMPLETO', 'COSTO_PROGETTO', 'IMPORTO_FINANZIAMENTO',
       'STATO_PROGETTO', 'TIPO_PROGETTO', 'CODICE_COMPONENTE',
       'CODICE_MISSIONE', 'COMPONENTE', 'INVESTIMENTO', 'MISSIONE'],
      dtype='object', length=113)
(273879, 113)


In [98]:
# sintesi in un solo block
# con P05 con tutte le colonne
leftnew_3_complete = p05.merge(p03, on='CIG',how='left')
leftnew_3_complete = leftnew_3_complete.drop_duplicates()
leftnew_34_complete = leftnew_3_complete.merge(p04, on='CIG', how='left')
leftnew_34_complete = leftnew_34_complete.drop_duplicates()
leftnew_341_complete = leftnew_34_complete.merge(p01, on='CIG', how='left')
leftnew_341_complete = leftnew_341_complete.drop_duplicates()
leftnew_3412_complete = leftnew_341_complete.merge(p02,how='left',left_on='CUP',right_on='CODICE_CUP')
leftnew_3412_complete = leftnew_3412_complete.drop_duplicates()

In [34]:
selezione = ['CIG', 'CUP', 'REGIONE', 'Regione', 'provincia', 'PROVINCIA', 'COMUNE',   'importo_complessivo_gara', 'CLASSE_IMPORTO', 'NOME_TEMATICA', 'CODICE_MISSIONE', 'MISSIONE', 'CODICE_COMPONENTE', 'COMPONENTE',  'INVESTIMENTO', 'cod_mis_premiale', 'misura_premiale', 'CATEGORIA',  'flag_quote', 'quota_femminile', 'quota_giovanile',
'cod_mot_deroga', 'mot_deroga', 'data_pubblicazione', 'anno_pubblicazione', 'FLAG_URGENZA', 'MOTIVO_URGENZA', 'ESITO']

In [35]:
p_basedati = leftnew_3412[selezione]

In [48]:
p_basedati[(p_basedati['REGIONE']!=p_basedati['Regione'].str.upper())&(p_basedati['Regione'].notna())&(p_basedati['REGIONE'].notna())]['REGIONE'].value_counts()

REGIONE
TRENTINO-ALTO ADIGE      4022
VALLE D'AOSTA             955
FRIULI-VENEZIA GIULIA     786
CAMPANIA                  616
LAZIO                     597
LOMBARDIA                 499
VENETO                    406
EMILIA-ROMAGNA            287
PUGLIA                    254
SICILIA                   247
PIEMONTE                  192
TOSCANA                   178
LIGURIA                    94
BASILICATA                 63
CALABRIA                   51
ABRUZZO                    27
SARDEGNA                   21
MOLISE                     18
MARCHE                     10
UMBRIA                      9
Name: count, dtype: int64

In [49]:
p_basedati.to_csv("basedati.csv", index = False, sep=';')

prima

In [99]:
# prima
# con tutti i campi p02, con tutti i campi P05
p07 = leftnew_3412_complete
# con tutti i campi p02, i campi di P05 sono già selezionati
p07_rid = leftnew_3412
# selezione di campi
p07_base = p07_rid[['CIG', 'importo_complessivo_gara','CLASSE_IMPORTO',
       'cod_mis_premiale', 'misura_premiale', 'CATEGORIA', 'flag_quote',
       'quota_femminile', 'quota_giovanile', 'cod_mot_deroga', 'mot_deroga',
       'CUP', 'REGIONE', 'PROVINCIA', 'COMUNE','ANNO_DECISIONE','data_pubblicazione', 'COSTO_PROGETTO', 'IMPORTO_FINANZIAMENTO', 'CODICE_MISSIONE', 'COMPONENTE',
       'INVESTIMENTO', 'MISSIONE']]

In [105]:
print(p07_base.shape)
print(p07_rid.shape)
print(p07.shape)

(273879, 23)
(273879, 56)
(273879, 113)


In [101]:
# è un problema di match coi comuni?
p07_rid[p07_rid['CIG']=="A0011137E1"]

,CIG,importo_complessivo_gara,CLASSE_IMPORTO,luogo_istat,provincia,Denominazione sovracomunale,Regione,data_pubblicazione,cod_mis_premiale,misura_premiale,...,DATA_GENERAZIONE_COMPLETO,COSTO_PROGETTO,IMPORTO_FINANZIAMENTO,STATO_PROGETTO,TIPO_PROGETTO,CODICE_COMPONENTE,CODICE_MISSIONE,COMPONENTE,INVESTIMENTO,MISSIONE
0,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
1,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
2,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
3,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
4,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
5,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA


In [103]:
p07_base.to_csv("tmp_finale_base.csv", index = False, sep=';')
p07_rid.to_csv("tmp_finale_ridotto.csv", index = False, sep=';')
p07.to_csv("tmp_finale.csv", index = False, sep=';')